# Environment setup
Please execute once to download the word list.

In [4]:
#!wget https://gist.githubusercontent.com/cfreshman/a03ef2cba789d8cf00c08f767e0fad7b/raw/5d752e5f0702da315298a6bb5a771586d6ff445c/wordle-answers-alphabetical.txt
#!wget https://gist.githubusercontent.com/cfreshman/cdcdf777450c5b5301e439061d29694c/raw/de1df631b45492e0974f7affe266ec36fed736eb/wordle-allowed-guesses.txt

--2022-02-02 18:47:52--  https://gist.githubusercontent.com/cfreshman/cdcdf777450c5b5301e439061d29694c/raw/de1df631b45492e0974f7affe266ec36fed736eb/wordle-allowed-guesses.txt
gist.githubusercontent.com (gist.githubusercontent.com) をDNSに問いあわせています... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
gist.githubusercontent.com (gist.githubusercontent.com)|185.199.111.133|:443 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 63941 (62K) [text/plain]
`wordle-allowed-guesses.txt' に保存中

wordle-allowed-gues 100%[===================>]  62.44K  --.-KB/s    in 0.01s   

2022-02-02 18:47:52 (6.30 MB/s) - `wordle-allowed-guesses.txt' へ保存完了 [63941/63941]



## Installing required modules

In [25]:
#conda install -y tqdm

# Module Imports

In [900]:
import tqdm.notebook as tqdm
from copy import copy
import random
from IPython.display import HTML
import sys

# Loading word list

In [177]:
answers = open("wordle-answers-alphabetical.txt", "r").read().split("\n")
guess   = open("wordle-allowed-guesses.txt", "r").read().split("\n")

In [939]:
MATCH_1 =  3
APPEAR_1 = 2
NEWCHR_1 = 4
MATCH_2 =  5
APPEAR_2 = 2
NEWCHR_2 = 1
def score(word, answers, matched=["-"] * 5, appeared="", discarded =""):
    word = word.lower()
    s = 0
    found = len([s for s in matched if s != "-"]) >= 4
    found = found or len([c for c in word if c in appeared]) == 5
    for i, c in enumerate(word):
        c = c.lower()
        for ans in answers:
            if ans[i] == c:
                s += MATCH_1 if not found else MATCH_2
            elif c in ans:
                s += APPEAR_1 if not found else APPEAR_2
            if c not in appeared and c not in discarded and c not in word[0:i]:
                s += NEWCHR_1 if not found else NEWCHR_2
    return s

In [940]:
def check_match(word, answer):
    matched = ["-"] * 5
    appeared = ""
    discarded = ""
    for i, a, b in zip(range(5), word, answer):
        if a == b:
            matched[i] = a.lower()
    for c in word:
        if c in answer:
            appeared += c.lower()
        else:
            discarded += c.lower()
    return matched, appeared, discarded

In [941]:
def guessing(trial, guess, matched, appeared, discarded):
    guess2 = []
    for ans in guess:
        rejected = False
        for p in range(len(matched)):
            if matched[p] != "-" and ans[p] != matched[p]:
                rejected = True
                break
        if not rejected:
            for c in appeared:
                if c not in ans:
                    rejected = True
                    break
        if not rejected:
            for c in discarded:
                if c in ans:
                    rejected = True
                    break
        if not rejected:
            guess2.append(ans)    
    
    scores = [score(word, answers, matched, appeared) for word in tqdm.tqdm(guess2)]
    wordbag = [a for a in zip(guess2, scores)]
    sorted_words=sorted(wordbag, key=lambda x: -x[1])
    return sorted_words, guess2

def merge(matched1, appeared1, discarded1, matched2, appeared2, discarded2):
    matched = ["-"] * len(matched1)
    for i in range(len(matched1)):
        if matched1[i] != "-":
            matched[i] = matched1[i].lower()
        elif matched2[i] != "-":
            matched[i] = matched2[i].lower()
    appeared = set(appeared1) | set(appeared2)
    discarded = set(discarded1) | set(discarded2)
    return matched, appeared, discarded

def meet(word, matched, appeared, discarded):
    word = word.lower()
    html = ""    
    def block(color, c):
        return "<span style='display: inline-block; text-align: center; color:#ffffff; font-weight: 700; margin: 2px; width: 24px; height: 24px; background:%s'>%s</span>"%(color, c.upper())
    for i, c in enumerate(word):
        if matched[i] == c:
            html += block("#00aa00", c)
        elif c in appeared:
            html += block("#aaaa00", c)
        else:
            html += block("#888888", c)
    display(HTML("<div>%s</div>"%html))

    for i in range(len(matched)):
        if matched[i] == "-":
            return False
    return True

# Extracting words of first step.

In [942]:
matched = ["-"]*5
appeared = ""
discarded = ""
guess2 = answers.copy()
first_words, first_guess = guessing(0, guess2, matched, appeared, discarded)
first_words = first_words

  0%|          | 0/2315 [00:00<?, ?it/s]

# Auto solver

In [951]:
policy = "best" # best or random

## Answer detemination

In [952]:
answer = random.choice(answers)
display(HTML("<h3 style='color: red'>word is '%s'</h3>"%answer.upper()))

In [953]:
matched = ["-"]*5
appeared = ""
discarded = ""
#guess2 = answers.copy()
sorted_words = first_words
guess2 = first_guess.copy()

if policy not in ["best", "random"]:
    print("Bad policy '%s'"%policy)
    sys.exit(1)

for trial in range(6):
    print("Trial #%d: "%(trial+1))
    if not sorted_words:
        sorted_words, guess2 = guessing(trial, guess2, matched, appeared, discarded)
    else:
        print("Result of first guess is pre-calcurated. skipped.")

    choice = None
    
    if policy == "best":
        choice = sorted_words[0][0]
    elif policy == "random":
        choice = random.choice(sorted_words)[0]
    
    matched2, appeared2, discarded2 = check_match(choice, answer)

    guess2.remove(choice)

    if meet(choice, matched2, appeared2, discarded2):
        break
    
    matched, appeared, discarded = merge(matched, appeared, discarded, matched2, appeared2, discarded2)
    print("candidates :", [s[0] for s in sorted_words[:5]])
#    print("Trial count %d: "%(trial+1), sorted_words[0][0], matched, appeared, discarded)
    sorted_words=None

Trial #1: 
Result of first guess is pre-calcurated. skipped.


candidates : ['stare', 'arose', 'raise', 'arise', 'irate']
Trial #2: 


  0%|          | 0/203 [00:00<?, ?it/s]

candidates : ['clone', 'opine', 'boule', 'guile', 'olden']
Trial #3: 


  0%|          | 0/14 [00:00<?, ?it/s]

candidates : ['biome', 'dopey', 'movie', 'video', 'oxide']
Trial #4: 


  0%|          | 0/1 [00:00<?, ?it/s]

# Manual Trial

In [755]:
matched = ["-"]*5
appeared = ""
discarded = ""
#guess2 = answers.copy()
sorted_words = first_words
guess2 = first_guess.copy()

## Trial loop

In [614]:
if not sorted_words:
    sorted_words, guess2 = guessing(trial, guess2, matched, appeared, discarded)

sorted_words[:5]

  0%|          | 0/5 [00:00<?, ?it/s]

[('shard', 53613),
 ('sharp', 53503),
 ('shark', 53272),
 ('spark', 53123),
 ('swarm', 52923)]

In [612]:
# Input result
matched2 = ["","","","",""]
appeared2 = ""
discarded2 = ""
meet(sorted_words[0][0], matched2, appeared2, discarded2)

False

In [613]:
guess2.remove(sorted_words[0][0])
matched, appeared, discarded = merge(matched, appeared, discarded, matched2, appeared2, discarded2)
print(sorted_words[0][0], matched, appeared, discarded)
sorted_words = None

scary ['s', '-', 'a', 'r', '-'] set() {'l', 't', 'y', 'n', 'e', 'c'}
